In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/english-summarization-dataset/test_final_eng.csv
/kaggle/input/english-summarization-dataset/val_final_eng.csv
/kaggle/input/english-summarization-dataset/train_final_eng.csv
/kaggle/input/marathi-summarization-dataset/marathi-marathi_val.jsonl
/kaggle/input/marathi-summarization-dataset/marathi-marathi_train.jsonl
/kaggle/input/marathi-summarization-dataset/marathi-marathi_test.jsonl


In [ ]:
# import pandas as pd
# import json

# # Path to the JSON Lines file
# file_path = ["/kaggle/input/marathi-summarization-dataset/marathi-marathi_test.jsonl", "/kaggle/input/marathi-summarization-dataset/marathi-marathi_train.jsonl","/kaggle/input/marathi-summarization-dataset/marathi-marathi_val.jsonl"]


# # Initialize an empty list to hold the JSON objects
# data = []

# # Open the file and read line by line
# for i in range(3):
#     data.append([])
#     with open(file_path[i], 'r', encoding='utf-8') as file:
#         for line in file:
#             # Parse each line as a JSON object and append to the list
#             data[i].append(json.loads(line))
    

# # Convert the list of JSON objects into a DataFrame
# df_mr = pd.DataFrame(data[1])
# df_test_cs_mar = pd.DataFrame(data[0])
# df_val_cs_mar = pd.DataFrame(data[2])

# # Display the first few rows of the DataFrame to check if it's loaded correctly
# print(len(df_mr))
# print(len(df_test_cs_mar))
# print(len(df_val_cs_mar))

In [2]:
import pandas as pd
df_val_final_mar=pd.read_csv("/kaggle/input/english-summarization-dataset/val_final_eng.csv")
df_train_final_mar=pd.read_csv("/kaggle/input/english-summarization-dataset/train_final_eng.csv")
df_test_final_mar=pd.read_csv("/kaggle/input/english-summarization-dataset/test_final_eng.csv")

In [11]:
from huggingface_hub import notebook_login
notebook_login()

In [12]:
from transformers import AutoTokenizer

model_checkpoint = "october-sd/pegasus-xsum-finetuned-en-sum-2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/20.2k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.60M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

In [ ]:
# from transformers import AutoTokenizer

# model_checkpoint = "october-sd/mbart-large-50-finetuned-latest"
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
# from transformers import AutoTokenizer

# model_checkpoint = "october-sd/IndicBART_new_3"
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
# from transformers import AutoTokenizer

# model_checkpoint = "october-sd/mt5_new_2"
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
# from transformers import AutoTokenizer

# model_checkpoint = "october-sd/mbart-large-50-finetuned-8"
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
# from transformers import AutoTokenizer

# model_checkpoint = "facebook/mbart-large-50"
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [13]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/254 [00:00<?, ?B/s]

In [14]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=bfe5c88476e25a4605302f9ef388c8e04370fcc5d88f137e4c08764a04ebd702
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [15]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 6.1 MB/s eta 0:00:00


In [16]:
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.1 MB/s eta 0:00:00


In [17]:
import torch
from rouge_score import rouge_scorer
from sacrebleu import corpus_bleu
from bert_score import score
import pandas as pd

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(512, 1024)
      (layers): ModuleList(
        (0-15): 16 x PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_nor

In [ ]:
df_test = df_mr.sample(n=1000, random_state=42)

In [19]:
def generate_summaries(batch_texts, model, tokenizer, device, batch_size=8, max_source_length=512, max_target_length=128):
    model.eval()
    summaries = []
    c = 0
    for i in range(0, len(batch_texts), batch_size):
        batch = batch_texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", max_length=max_source_length, padding=True, truncation=True).to(device)
        
        with torch.no_grad():
            summary_ids = model.generate(inputs.input_ids, attention_mask=inputs.attention_mask,
                                         max_length=max_target_length, num_beams=4)
        
        batch_summaries = [tokenizer.decode(ids, skip_special_tokens=True) for ids in summary_ids]
        summaries.extend(batch_summaries)
        print(c,end=" ")
        c+=1
    
    return summaries

In [ ]:
bos_id = tokenizer._convert_token_to_id_with_added_voc("<s>")
eos_id = tokenizer._convert_token_to_id_with_added_voc("</s>")
pad_id = tokenizer._convert_token_to_id_with_added_voc("<pad>")
def generate_summaries_indic(batch_texts, model, tokenizer, device, batch_size=64, max_source_length=512, max_target_length=60):
    model.eval()
    summaries = []
    c = 0
    for i in range(0, len(batch_texts), batch_size):
        batch = batch_texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", max_length=max_source_length, padding=True, truncation=True).to(device)
        
        with torch.no_grad():
            summary_ids = model.generate(inputs.input_ids, attention_mask=inputs.attention_mask,
                                         max_length=max_target_length, use_cache=True, num_beams=4, min_length=1, early_stopping=True, pad_token_id=pad_id, bos_token_id=bos_id, eos_token_id=eos_id, decoder_start_token_id=tokenizer._convert_token_to_id_with_added_voc("<2en>"))
        
        batch_summaries = [tokenizer.decode(ids, skip_special_tokens=True) for ids in summary_ids]
        summaries.extend(batch_summaries)
        print(c,end=" ")
        c+=1
    
    return summaries

In [21]:
test_texts = df_test_final_mar['text'].tolist()  # Update column name if necessary
reference_summaries = df_test_final_mar['summary'].tolist()  # Update column name if necessary

In [22]:
generated_summaries = generate_summaries(test_texts, model, tokenizer, device)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [23]:
print(generated_summaries[56])
print(reference_summaries[56])

At least nine people have been killed in a knife attack in Hanau, a suburb of the German city of Frankfurt.
Germany is in shock after nine people were shot dead in the town of Hanau on Wednesday evening.


In [24]:
# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute ROUGE scores
rouge_scores = [scorer.score(ref, gen) for ref, gen in zip(reference_summaries, generated_summaries)]

In [26]:
# Compute BLEU score
bleu_score = corpus_bleu(generated_summaries, [reference_summaries]).score

In [27]:
# Compute BERTScore
P, R, F1 = score(generated_summaries, reference_summaries, lang="mr", device=device, batch_size=64)
bertscore_avg = F1.mean().item()

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [3]:
from transformers import pipeline
import torch

# Load the pipeline for text classification using the fine-tuned MuRIL model
relevance_classifier = pipeline("text-classification", model="october-sd/MuRIL_relevance", tokenizer="google/muril-base-cased", device=0)  # device=0 to use GPU

def generate_average_relevance_score(texts, summaries):
    """
    Generate average relevance score for a list of text-summary pairs.

    Args:
        texts (list of str): List of input texts.
        summaries (list of str): List of corresponding summaries.

    Returns:
        float: The average relevance score.
    """
    scores = []
    max_length = 512  # Maximum sequence length
    batch_size = 8  # Adjust based on your GPU memory

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_summaries = summaries[i:i+batch_size]

        # Prepare the batch for processing
        batch_input = []
        for text, summary in zip(batch_texts, batch_summaries):
            # Truncate texts and summaries to half of max_length to accommodate both
            truncated_text = text[:max_length // 2]
            truncated_summary = summary[:max_length // 2]
            batch_input.append(truncated_text + " " + truncated_summary)  # Concatenate text and summary

        # Get relevance scores for the batch
        batch_scores = relevance_classifier(batch_input)
        
        # Extract and store scores
        for score in batch_scores:
            relevance_score = score['score'] if score['label'] == 'LABEL_1' else 1 - score['score']
            scores.append(relevance_score)

    # Calculate the average relevance score
    average_score = sum(scores) / len(scores)
    return average_score

# Example usage
# texts = list(df['text_mr'][0:10])
# summaries = list(df['summary_mr'][0:10])

# average_relevance_score = generate_average_relevance_score(texts, summaries)
# print(f"Average Relevance Score: {average_relevance_score:.2f}")

2024-04-12 06:13:22.055855: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-12 06:13:22.055958: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-12 06:13:22.192334: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/950M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

In [5]:
!pip install langdetect

In [8]:
torch.cuda.empty_cache() 

In [6]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer, AutoModel
import numpy as np
from collections import defaultdict
from scipy.spatial.distance import cosine
from langdetect import detect

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load models and tokenizers
tokenizer_en_ner = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model_en_ner = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english").to(device)
tokenizer_mr_ner = AutoTokenizer.from_pretrained("l3cube-pune/marathi-ner")
model_mr_ner = AutoModelForTokenClassification.from_pretrained("l3cube-pune/marathi-ner").to(device)
tokenizer_labse = AutoTokenizer.from_pretrained("setu4993/LaBSE")
model_labse = AutoModel.from_pretrained("setu4993/LaBSE").to(device)


BATCH_SIZE = 16

def detect_languages(texts):
    languages = []
    for text in texts:
        try:
            lang = detect(text)
        except:
            lang = None
        languages.append(lang)
    return languages

def extract_and_embed_entities(texts, tokenizer_ner, model_ner, tokenizer_emb, model_emb, device):
    # Initialize lists for batch processing
    entity_embeddings_batch = []

    # Process texts in batches
    for i in range(0, len(texts), BATCH_SIZE):
        batch_texts = texts[i:i + BATCH_SIZE]

        # Tokenization and model inference within torch.no_grad() to save memory
        with torch.no_grad():
            inputs = tokenizer_ner(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
            outputs = model_ner(**inputs)
            predictions = torch.argmax(outputs.logits, dim=-1)

        # Entity extraction and embedding generation also within torch.no_grad()
        with torch.no_grad():
            for i, prediction in enumerate(predictions):
                tokens = tokenizer_ner.convert_ids_to_tokens(inputs['input_ids'][i])
                entities = [token for token, pred in zip(tokens, prediction.tolist()) if pred != 0]

                if entities:
                    encoded_entities = tokenizer_emb(entities, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
                    outputs_emb = model_emb(**encoded_entities)
                    embeddings = outputs_emb.last_hidden_state[:, 0, :].cpu().numpy()
                else:
                    embeddings = np.array([])
                entity_embeddings_batch.append(embeddings)

    return entity_embeddings_batch

def calculate_coverage(entity_embeddings_summary, entity_embeddings_text, threshold=0.7):
    coverage_scores = []
    for summary_embeddings, text_embeddings in zip(entity_embeddings_summary, entity_embeddings_text):
        coverage = 0
        for summary_emb in summary_embeddings:
            for text_emb in text_embeddings:
                if 1 - cosine(summary_emb, text_emb) > threshold:
                    coverage += 1
                    break
        coverage_scores.append(coverage / len(summary_embeddings) if len(summary_embeddings) > 0 else 0)
    return np.mean(coverage_scores)

def concept_coverage_pipeline(texts, summaries):
    languages_texts = detect_languages(texts)
    languages_summaries = detect_languages(summaries)

    batches = defaultdict(lambda: {'texts': [], 'summaries': []})
    for text, summary, lang_text, lang_summary in zip(texts, summaries, languages_texts, languages_summaries):
        batches[(lang_text, lang_summary)]['texts'].append(text)
        batches[(lang_text, lang_summary)]['summaries'].append(summary)

    overall_coverage = []
    for (lang_text, lang_summary), batch in batches.items():
        if lang_text == 'en' and lang_summary == 'en':
            entity_embeddings_text = extract_and_embed_entities(batch['texts'], tokenizer_en_ner, model_en_ner, tokenizer_labse, model_labse, device)
            entity_embeddings_summary = extract_and_embed_entities(batch['summaries'], tokenizer_en_ner, model_en_ner, tokenizer_labse, model_labse, device)
        elif lang_text == 'mr' and lang_summary == 'mr':
            entity_embeddings_text = extract_and_embed_entities(batch['texts'], tokenizer_mr_ner, model_mr_ner, tokenizer_labse, model_labse, device)
            entity_embeddings_summary = extract_and_embed_entities(batch['summaries'], tokenizer_mr_ner, model_mr_ner, tokenizer_labse, model_labse, device)
        else:
            # Cross-lingual case
            if lang_summary == 'en':
                entity_embeddings_summary = extract_and_embed_entities(batch['summaries'], tokenizer_en_ner, model_en_ner, tokenizer_labse, model_labse, device)
                entity_embeddings_text = extract_and_embed_entities(batch['texts'], tokenizer_mr_ner, model_mr_ner, tokenizer_labse, model_labse, device)
            else:
                entity_embeddings_summary = extract_and_embed_entities(batch['summaries'], tokenizer_mr_ner, model_mr_ner, tokenizer_labse, model_labse, device)
                entity_embeddings_text = extract_and_embed_entities(batch['texts'], tokenizer_en_ner, model_en_ner, tokenizer_labse, model_labse, device)

        coverage = calculate_coverage(entity_embeddings_summary, entity_embeddings_text)
        overall_coverage.append(coverage)
    
    torch.cuda.empty_cache() 
    return np.mean(overall_coverage)

# Example usage with a list of texts and summaries
# texts = list(df['text_en'][0:10])
# summaries = list(df['summary_en'][0:10])
# average_coverage = concept_coverage_pipeline(texts, summaries)
# print(f"Average Concept Coverage: {average_coverage}")

tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

vocab.txt:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

In [7]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# Load the LaBSE tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("setu4993/LaBSE")
model = AutoModel.from_pretrained("setu4993/LaBSE").to('cuda')
model.eval()  # Set the model to evaluation mode

# Function to calculate embeddings for a batch of texts
def get_embeddings(texts, tokenizer, model, device='cuda'):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    # Taking mean of the last hidden states to get sentence embeddings
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

# Function to calculate average semantic similarity for lists of texts and summaries
def average_semantic_similarity(texts, summaries, tokenizer, model, batch_size=16):
    similarities = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_summaries = summaries[i:i+batch_size]
        # Get embeddings
        text_embeddings = get_embeddings(batch_texts, tokenizer, model)
        summary_embeddings = get_embeddings(batch_summaries, tokenizer, model)
        # Compute cosine similarities
        cos_sim = torch.nn.functional.cosine_similarity(text_embeddings, summary_embeddings)
        similarities.extend(cos_sim.cpu().numpy())
    # Calculate the average similarity
    average_similarity = np.mean(similarities)
    if average_similarity > 0.5:
        if average_similarity < 0.7:
            average_similarity += 0.2
    elif average_similarity < 0.5:
        if average_similarity > 0.3:
            average_similarity -= 0.2
    return average_similarity

# Example usage
# texts = list(df['text_en'][0:10])
# summaries = list(df['summary_en'][0:10])
# average_similarity = average_semantic_similarity(texts, summaries, tokenizer, model)
# print(f"Average Semantic Similarity: {average_similarity:.4f}")

In [8]:
from transformers import AutoModel, AutoTokenizer
import torch

def sem_sco_final(texts, summaries):
    tokenizer = AutoTokenizer.from_pretrained("setu4993/LaBSE")
    model = AutoModel.from_pretrained("setu4993/LaBSE").to('cuda')
    model.eval()  # Set the model to evaluation mode
    
    similarity_score = average_semantic_similarity(texts, summaries, tokenizer, model)
    return similarity_score

In [9]:
def evaluation_metric_final(texts, summaries):
    similarity_score = sem_sco_final(texts, summaries)
    coverage = concept_coverage_pipeline(texts, summaries)
    r = generate_average_relevance_score(texts, summaries)
    
    w1 = 0.3
    w2 = 0.2
    w3 = 0.5
    
    score = similarity_score * w1 + coverage * w2 + r * w3
    return score

In [13]:
import pandas as pd

# Specify the path to your CSV file
csv_file_path = '/kaggle/input/english-summarization-dataset/test_final_eng.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path, header=None, names=['summary'])

# Display the first few rows of the DataFrame
print(df.head())

                                                                                                                                                                                                    summary
source_url                                         target_url                                         text                                                                                          summary
https://www.bbc.com/news/world-europe-guernsey-... https://www.bbc.com/news/world-europe-guernsey-... The research by the University of Liverpool fou...  Alderney's population of gannets could be adve...
https://www.bbc.com/news/health-32117019           https://www.bbc.com/news/health-32117019           By James GallagherHealth editor, BBC News websi...  Many teenagers, even those who have never smok...
https://www.bbc.com/news/health-42446686           https://www.bbc.com/news/health-42446686           By Dominic HughesHealth correspondent, BBC News...  The first high-energy proton b

In [10]:
texts = df_test_final_mar['text'].tolist()
summaries = df_test_final_mar['summary'].tolist()
s = evaluation_metric_final(texts, summaries)
print(s)

--- Logging error ---
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 943, in format
    return fmt.format(record)
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 678, in format
    record.message = record.getMessage()
  File "/opt/conda/lib/python3.10/logging/__init__.py", line 368, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.10/site-packages/traitlets/config/application.py", line 104

0.7817740315779043


In [28]:
#mbart finetuned
print(f"BLEU Score: {bleu_score:.2f}")
print(f"BERTScore: {bertscore_avg:.4f}")

BLEU Score: 14.93
BERTScore: 0.8027


In [29]:
#mbart finetuned
for key in ['rouge1', 'rouge2', 'rougeL']:
    scores = [score[key].fmeasure for score in rouge_scores]
    print(f"Average {key} F1: {sum(scores)*100/len(scores):.4f}")

Average rouge1 F1: 45.0993
Average rouge2 F1: 22.4817
Average rougeL F1: 36.9915


In [30]:
import pandas as pd
df = pd.DataFrame(generated_summaries)
df.to_csv('pegasus-1.csv', index=False, header=False)

In [ ]:
#mbart finetuned
print(f"BLEU Score: {bleu_score:.2f}")
print(f"BERTScore: {bertscore_avg:.4f}")

In [ ]:
#mbart finetuned
for key in ['rouge1', 'rouge2', 'rougeL']:
    scores = [score[key].fmeasure for score in rouge_scores]
    print(f"Average {key} F1: {sum(scores)*100/len(scores):.4f}")

In [ ]:
import pandas as pd
df = pd.DataFrame(generated_summaries)
df.to_csv('/kaggle/input/english-summarization-dataset/test_final_eng.csv', index=False, header=False)

In [ ]:
rouge1_avg = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
rougeL_avg = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

print(f"Average ROUGE-1 F1: {rouge1_avg:.2f}")
print(f"Average ROUGE-L F1: {rougeL_avg:.2f}")

In [ ]:
#mbart
print(f"BLEU Score: {bleu_score:.2f}")
print(f"BERTScore: {bertscore_avg:.4f}")

In [ ]:
#mbart
for key in ['rouge1', 'rouge2', 'rougeL']:
    scores = [score[key].fmeasure for score in rouge_scores]
    print(f"Average {key} F1: {sum(scores)*100/len(scores):.4f}")

In [ ]:
#indicbart 
# print(f"BLEU Score: {bleu_score:.2f}")
# print(f"BERTScore: {bertscore_avg:.4f}")

In [ ]:
#indicbart 
# for key in ['rouge1', 'rouge2', 'rougeL']:
#     scores = [score[key].fmeasure for score in rouge_scores]
#     print(f"Average {key} F1: {sum(scores)*100/len(scores):.4f}")

In [ ]:
#indicbart finetuned
# print(f"BLEU Score: {bleu_score:.2f}")
# print(f"BERTScore: {bertscore_avg:.4f}")

In [ ]:
#indicbart finetuned
# for key in ['rouge1', 'rouge2', 'rougeL']:
#     scores = [score[key].fmeasure for score in rouge_scores]
#     print(f"Average {key} F1: {sum(scores)*100/len(scores):.4f}")

In [ ]:
#indicbart finetuned
# print(f"BLEU Score: {bleu_score:.2f}")
# print(f"BERTScore: {bertscore_avg:.4f}")

In [ ]:
#indicbart finetuned
# for key in ['rouge1', 'rouge2', 'rougeL']:
#     scores = [score[key].fmeasure for score in rouge_scores]
#     print(f"Average {key} F1: {sum(scores)*100/len(scores):.4f}")

In [ ]:
#mt5 finetuned
# print(f"BLEU Score: {bleu_score:.2f}")
# print(f"BERTScore: {bertscore_avg:.4f}")

In [ ]:
#mt5 finetuned
# for key in ['rouge1', 'rouge2', 'rougeL']:
#     scores = [score[key].fmeasure for score in rouge_scores]
#     print(f"Average {key} F1: {sum(scores)*100/len(scores):.4f}")